In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

1. **Dealing with Missing Values**
2. **Removing Duplicates**
3. **Dealing With Inconsistancy**
3. **Reshaping Data**

In [ ]:
# Data Loading
df = pd.read_csv('/kaggle/input/bengaluru-house-price-data/Bengaluru_House_Data.csv')

In [ ]:
# Making a copy pf dataframe as data
data=df.copy()

In [ ]:
# Data Info
data.info()

In [ ]:
data.isna().sum()/data.shape[0]*100

**Insights from info and looking at data**
* There are some null Values in out Data set.
* Irrelevant data types of columns- total-sqft, bath, balcony
* Availability column - Non Consistant data
* Size Column- need to extract numeric data
* Total-sqft column- have range value and some in other unit(sq. meter...etc).
* Location column- One missing data
* Society Column-- 41% data missing(Lot of data is missing)

In [ ]:
# Dealing Missing Values.
data.isna().sum()

In [ ]:
# Society Column-
data['society'].fillna("Info Not available",inplace = True)

In [ ]:
# Location Column--
data[data.location.isna()]

In [ ]:
# Looking for same society
data[data.society=='Grare S']

Above for Society Grare S location is Anantapura.

In [ ]:
data.location.fillna('Anantapura',inplace=True)

In [ ]:
# Size Column--
data[data['size'].isna()]

In [ ]:
data[data.society=='Orana N']

In [ ]:
data.area_type.unique()

In [ ]:
# Will look in missing Value later.

In [ ]:
# Total Sqft column- dealing with range value data and other sq. unit data.

In [ ]:
def preprocess_total_sqft(my_list):
    if len(my_list) == 1:
        
        try:
            return float(my_list[0])
        except:
            strings = ['Sq. Meter', 'Sq. Yards', 'Perch', 'Acres', 'Cents', 'Guntha', 'Grounds']
            split_list = re.split('(\d*.*\d)', my_list[0])[1:]
            area = float(split_list[0])
            type_of_area = split_list[1]
            
            if type_of_area == 'Sq. Meter':
                area_in_sqft = area * 10.7639
            elif type_of_area == 'Sq. Yards':
                area_in_sqft = area * 9.0
            elif type_of_area == 'Perch':
                area_in_sqft = area * 272.25
            elif type_of_area == 'Acres':
                area_in_sqft = area * 43560.0
            elif type_of_area == 'Cents':
                area_in_sqft = area * 435.61545
            elif type_of_area == 'Guntha':
                area_in_sqft = area * 1089.0
            elif type_of_area == 'Grounds':
                area_in_sqft = area * 2400.0
            return float(area_in_sqft)
        
    else:
        return (float(my_list[0]) + float(my_list[1]))/2.0

In [ ]:
data['total_sqft'] = data.total_sqft.str.split('-').apply(preprocess_total_sqft)

In [ ]:
data.groupby('size')['total_sqft'].mean()

In [ ]:
data['size'].unique()

1 BHK- 1 Bedroom hall and Kitchen

1 RK- 1 Room(Bedroom+hall combined) and Kitchen

In [ ]:
data['size']

In [ ]:
data.head()

In [ ]:
data.isnull().sum()

In [ ]:
data[data['size'].isna()]

# Filling size nan value--
* Taking total_sqft and comparing it with mean of groupby on size data and fill with closest one.

In [ ]:
sqft_list = data.groupby('size')['total_sqft'].mean()

In [ ]:
type(sqft_list)

In [ ]:
def closest_size(K):  
    idx = (np.abs(sqft_list - K)).argmin() 
    return sqft_list.index[idx]

In [ ]:
missing_size = data[data['size'].isna()]['total_sqft'].apply(closest_size)

In [ ]:
missing_size.shape

In [ ]:
missing_size

In [ ]:
data.iloc[missing_size.index,3]=missing_size.values

In [ ]:
# for i,j in zip(data['size'],data['total_sqft']):
#     if ~(type(i)==str):
#         i = closest(j)

In [ ]:
data.isnull().sum()

In [ ]:
# making three column-- bedroom, hall, kitchen
# Bedroom-
data['bedroom'] = data['size'].apply(lambda x: x.split(' ')[0] if isinstance(x,str) else np.nan)
data['bedroom'] = data['bedroom'].astype(float)


# Hall-
def isHall(x):
    if type(x)==str:
        tokens = x.split(' ')
        if tokens[1]=='BHK':
            return 1
        else:
            return 0
        
        
# Kitchen
def isKitchen(x):
    if type(x)==str:
        tokens = x.split(' ')
        if (tokens[1]=='BHK')| (tokens[1]=='RK'):
            return 1
        else:
            return 0

In [ ]:
data['hall']=data['size'].apply(isHall,)
data['kitchen']=data['size'].apply(isKitchen)

In [ ]:
# Two column missing value- Bath and balcony.
# can be filled with mode or as the above apraoch for size(based on sqft available)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
data.groupby('bath')['bedroom'].mean()

In [ ]:
# Bath
bedroom_list_bath = data.groupby('bath')['bedroom'].mean()

In [ ]:
df[df.bath==18]

In [ ]:
bedroom_list_bath

In [ ]:
data[data.bath==18]

In [ ]:
def closest_bath(K):  
    idx = (np.abs(bedroom_list_bath - K)).argmin() 
    return bedroom_list_bath.index[idx]

In [ ]:
missing_bath = data[data['bath'].isna()]['bedroom'].apply(closest_bath)

In [ ]:
missing_bath.shape

In [ ]:
missing_bath

In [ ]:
data.iloc[missing_bath.index,6]=missing_bath.values

In [ ]:
# Balcony
bedroom_list_balcony = data.groupby('balcony')['bedroom'].mean()

In [ ]:
def closest_balcony(K):  
    idx = (np.abs(bedroom_list_balcony - K)).argmin() 
    return bedroom_list_balcony.index[idx]

In [ ]:
missing_balcony = data[data['balcony'].isna()]['bedroom'].apply(closest_balcony)

In [ ]:
data.iloc[missing_balcony.index, 7]=missing_balcony.values

In [ ]:
data.isnull().sum()

In [ ]:
data.isnull().sum()

In [ ]:
data.head()

In [ ]:
data.describe()

In [ ]:
data[data.bath>10]

In [ ]:
data.iloc[344]

In [ ]:
df.loc[344]

In [ ]:
df[df.bath>10]

In [ ]:
data.isnull().sum()

In [ ]:
data.info()

In [ ]:
# No need to have size- as all data has been extracted in bedroom hall and kitchen column.
data.drop('size',axis=1 ,inplace=True)

In [ ]:
data.info()

In [ ]:
data.area_type.unique()

In [ ]:
data.society.nunique()

In [ ]:
data.location.nunique()

In [ ]:
plt.plot(data.location,data.price)

In [ ]:
data

In [ ]:
data.availability.unique()

## **Labeling- area_type column**
4-unique categories

In [ ]:
data.area_type.value_counts()

In [ ]:
replace_area_type = {'Super built-up  Area': 0, 'Built-up  Area': 1, 'Plot  Area': 2, 'Carpet  Area': 3}
data['area_type'] = data.area_type.map(replace_area_type)

**Converting the categorical values in the column 'availability' into 3 categories.**
* Ready to move
* Immediate Possession
* Others

In [ ]:
def replacing_availabilty(string):
    if string == 'Ready To Move':
        return 0
    elif string == 'Immediate Possession':
        return 1
    else:
        return 2

In [ ]:
data['availability'] = data.availability.apply(replacing_availabilty)

In [ ]:
data.head()

## **Labelling the column 'location'**

Using LabelEncoder to convert locations into numerals.

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
loc_encoder = LabelEncoder()
loc_encoder.fit(data['location'])
data['location'] = loc_encoder.transform(data['location'])

In [ ]:
data.head()

## **Labelling the column 'society'**

Using LabelEncoder to convert locations into numerals.

In [ ]:
loc_encoder = LabelEncoder()
loc_encoder.fit(data['society'])
data['society'] = loc_encoder.transform(data['society'])

In [ ]:
data.head()

In [ ]:
data.info()